# Homework 4: Deep Convolutional Generative Adversarial Network (DCGAN) (100 points)

In this assignment, we're going to build and train a DCGAN model to generate synthetic pokemon images.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
import torchvision

from torch import nn
from d2l import torch as d2l
from tqdm import tqdm

# Loading Data (0 points)

This is an example of using a built-in torchvision dataloader for a vision dataset, no code needed here.

The original data can be retrieved from: [Kaggle - Pokemon Image Dataset](https://www.kaggle.com/datasets/kvpratama/pokemon-images-dataset)

**Please note** that you will want to unzip the dataset (in `assets/data/pokemon.zip`) before you run this code for the first time. As the dataset is large, you may expect it to take about twenty minutes. 

In [ ]:
import zipfile
from io import BytesIO
import torchvision.transforms as transforms

# Load the zipped file into memory
with open('assets/data/pokemon.zip', 'rb') as f:
    zip_bytes = BytesIO(f.read())

# Open the zip file using the ZipFile module
with zipfile.ZipFile(zip_bytes, 'r') as zip_ref:
    # Extract the contents of the zip file to a temporary directory
    zip_ref.extractall('assets/data/pokemon_temp')

# Load the data using the ImageFolder function, pointing to the temporary directory
# pokemon= torchvision.datasets.ImageFolder('assets/data/pokemon_temp', transform=transforms.ToTensor())
pokemon= torchvision.datasets.ImageFolder('assets/data/pokemon_temp')

# Delete the temporary directory
# shutil.rmtree('assets/data/pokemon_temp')

In [ ]:
# # d2l.DATA_HUB['pokemon'] = (d2l.DATA_URL + 'pokemon.zip',
# #                            'c065c0e2593b8b161a2d7873e42418bf6a21106c')

# # data_dir = d2l.download_extract('pokemon', folder='assets/data')

# pokemon = torchvision.datasets.ImageFolder('assets/data/pokmeon')

In [ ]:
batch_size = 256

# the pipeline applied to images: resizing, scaling, normalizing
transformer = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(0.5, 0.5)])
pokemon.transform = transformer

# build data loader
data_iter = torch.utils.data.DataLoader(
    pokemon, batch_size=batch_size, shuffle=True,
    num_workers=3) 

In [ ]:
# Visualize a batch of data
d2l.set_figsize((4, 4))
for X, y in data_iter:
    imgs = X[0:20, :, :, :].permute(0, 2, 3, 1) / 2 + 0.5
    d2l.show_images(imgs, num_rows=4, num_cols=5)
    break

# Question 1: Build the Generator

In this part, you will build the generator of a DCGAN. Each generator is composed of several generator blocks.

Each generator block should include a convolutional layer, batch normalization, and an activation function.

The generator then consists of 4 blocks with different input/output channels, a convolutional layer and an activation function.

Finally, we will put the blocks into an nn.Sequential module as we did in Homework 2.

You can follow the architecture proposed in the original paper: [Unsupervised Representation Learning
with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/pdf/1511.06434.pdf)

In [ ]:
class G_block(nn.Module):
    def __init__(self, out_channels,in_channels=3, kernel_size=4, stride=2, padding=1, normalize=True):
        super(G_block, self).__init__()
        layers = []
        layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))
        if normalize:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

In [ ]:
net_G = nn.Sequential(
    G_block(100, 512, normalize=False),  # input: (batch_size, 100, 1, 1), output: (batch_size, 512, 4, 4)
    G_block(512, 256),  # input: (batch_size, 512, 4, 4), output: (batch_size, 256, 8, 8)
    G_block(256, 128),  # input: (batch_size, 256, 8, 8), output: (batch_size, 128, 16, 16)
    G_block(128, 64),  # input: (batch_size, 128, 16, 16), output: (batch_size, 64, 32, 32)
    nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),  # input: (batch_size, 64, 32, 32), output: (batch_size, 3, 64, 64)
    nn.Tanh()
)

In [ ]:
# assert statements to help check your solutions 

gb=G_block(100)
conv_num=0
batch_norm=0
for m in gb.modules():
    conv_num+=(type(m)==nn.ConvTranspose2d)
    batch_norm+=(type(m)==nn.BatchNorm2d)
assert conv_num==1, "Question 1: Number of 2D transposed convolution operators applied in G_block class does not match expected"
assert batch_norm==1, "Question 1: Number of batch normalizations applied does not match expected"



gblock_num=0
conv_num=0
for m in net_G:
    gblock_num+=(type(m)==G_block)
    conv_num+=(type(m)==nn.ConvTranspose2d)
assert gblock_num==4, "Question 1: Number of generator blocks does not match expected"
assert conv_num==1, "Question 1: Number of 2D transposed convolution operators applied in DCGAN generator does not match expected"



# Question 2: Build the Discriminator

In this question, you will build the generator of a DCGAN. Each generator is composed of several generator blocks.

Each generator block should include a convolutional layer, batch normalization, and an activation function.

The generator then consists of 4 blocks with different input/output channels, and a convolutional layer.

Finally, we will put the blocks into an nn.Sequential module as we did in Homework 2.

You can follow the architecture proposed in the original paper: [Unsupervised Representation Learning
with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/pdf/1511.06434.pdf)

In [ ]:
class D_block(nn.Module):
    def __init__(self, out_channels, in_channels=3, kernel_size=4, strides=2,
                 padding=1, alpha=0.2, **kwargs):
        super(D_block, self).__init__(**kwargs)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(alpha)


    def forward(self, X):
        out = self.conv(X)
        out = self.bn(out)
        out = self.activation(out)
        return out

In [ ]:
net_D = nn.Sequential(
    # input size: (3, 64, 64)
    D_block(64, in_channels=3), # output size: (64, 32, 32)
    D_block(128),                # output size: (128, 16, 16)
    D_block(256),                # output size: (256, 8, 8)
    D_block(512),                # output size: (512, 4, 4)
    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    nn.Sigmoid()  # Output: (1, 1, 1)
)

In [ ]:
# assert statements to help check your solutions

db=D_block(100)
conv_num=0
batch_norm=0
for m in db.modules():
    conv_num+=(type(m)==nn.Conv2d)
    batch_norm+=(type(m)==nn.BatchNorm2d)
assert conv_num==1, "Question 2: Number of 2D transposed convolution operators applied in D_block class does not match expected"
assert batch_norm==1, "Question 2: Number of batch normalizations applied does not match expected"



dblock_num=0
conv_num=0
for m in net_D:
    dblock_num+=(type(m)==D_block)
    conv_num+=(type(m)==nn.Conv2d)
assert dblock_num==4, "Question 2: Number of discriminator blocks does not match expected"
assert conv_num==1, "Question 2: Number of 2D transposed convolution operators applied in DCGAN discriminator does not match expected"


# Train the Network

Since GANs can be hard to train, we provide the full training code, so no code is needed here. But feel free to tune the network yourself. Please don’t remove the print statement included in the ``train`` function, as the instructional team will be using it to check whether your loss decreases monotonically or not.

You should expect to find that the loss of the generator generally increases while the loss of the discriminator decreases over the epochs. The output images will potentially `real`.

In [ ]:
def train(net_D, net_G, data_iter, num_epochs, lr, latent_dim,
          device=d2l.try_gpu()):
    loss = nn.BCEWithLogitsLoss(reduction='sum')
    for w in net_D.parameters():
        nn.init.normal_(w, 0, 0.02)
    for w in net_G.parameters():
        nn.init.normal_(w, 0, 0.02)
    net_D, net_G = net_D.to(device), net_G.to(device)
    trainer_hp = {'lr': lr, 'betas': [0.5, 0.999]}
    trainer_D = torch.optim.Adam(net_D.parameters(), **trainer_hp)
    trainer_G = torch.optim.Adam(net_G.parameters(), **trainer_hp)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss',
                            xlim=[1, num_epochs], nrows=2, figsize=(5, 5),
                            legend=['discriminator', 'generator'])
    animator.fig.subplots_adjust(hspace=0.3)
    for epoch in tqdm(range(1, num_epochs + 1)):
        # Train one epoch
        timer = d2l.Timer()
        metric = d2l.Accumulator(3)  # loss_D, loss_G, num_examples
        for X, _ in data_iter:
            batch_size = X.shape[0]
            Z = torch.normal(0, 1, size=(batch_size, latent_dim, 1, 1))
            X, Z = X.to(device), Z.to(device)
            metric.add(d2l.update_D(X, Z, net_D, net_G, loss, trainer_D),
                       d2l.update_G(Z, net_D, net_G, loss, trainer_G),
                       batch_size)
        # Show generated examples
        Z = torch.normal(0, 1, size=(21, latent_dim, 1, 1), device=device)
        # Normalize the synthetic data to N(0, 1)
        fake_x = net_G(Z).permute(0, 2, 3, 1) / 2 + 0.5
        imgs = torch.cat([
            torch.cat([fake_x[i * 7 + j].cpu().detach()
                       for j in range(7)], dim=1)
            for i in range(len(fake_x) // 7)], dim=0)
        animator.axes[1].cla()
        animator.axes[1].imshow(imgs)
        # Show the losses
        loss_D, loss_G = metric[0] / metric[2], metric[1] / metric[2]
        animator.add(epoch, (loss_D, loss_G))
    print(f'loss_D {loss_D:.3f}, loss_G {loss_G:.3f}, '
          f'{metric[2] / timer.stop():.1f} examples/sec on {str(device)}')

In [ ]:
# Train the network on GPU
# See how the loss of D and G change over time, and how the final image generations look like.
latent_dim, lr, num_epochs = 100, 0.005, 20
train(net_D, net_G, data_iter, num_epochs, lr, latent_dim)